# 8. fit with tf.data
이전 시간 (7. tf.data 사용하여 image & make batch.ipynb)에서 한 tf.data를 fit.generator에 넣어보는 작업 진행

### 0. 모듈 불러오기

In [1]:
from glob import glob

import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import datasets

import matplotlib.pyplot as plt
%matplotlib inline

### 1. hyperparameter 설정

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

### 2. 모델 설정 (Build model)

In [3]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

### 3. Data PreProgress 
(data generator사용하지 않는) 이전 시간에 했던 과정 반복 + 더 이어서

In [4]:
train_paths = glob('/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/*.png')
test_paths = glob('/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/*.png')

In [5]:
train_paths[:10]

['/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/32270_deer.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/21851_cat.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/48309_deer.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/33547_truck.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/45202_automobile.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/2789_bird.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/24517_horse.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/15193_frog.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/10817_ship.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/49897_ship.png']

In [6]:
test_paths[:10]

['/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/2598_cat.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/7195_horse.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/6171_deer.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/2586_horse.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/8372_automobile.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/3541_bird.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/332_cat.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/4811_bird.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/2255_automobile.png',
 '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/7418_ship.png']

In [7]:
#image, label 함께 가져오는 것도 이전 시간에 했던 과정 반복
#일단 train에서 첫번째꺼 하나만 해보기
#path = train_paths[0]
#path

In [9]:
def get_class_name(path):
    fname = tf.strings.split(path, '_')[-1]
    fname
    label_name = tf.strings.regex_replace(fname, '.png', '')
    return label_name

classes = [get_class_name(path) for path in train_paths]
classes = tf.unique(classes)


def onehot_encoding(label_name):
    classes = get_class_name(path)
    onehot = tf.cast(label_name == classes, tf.uint8)
    return onehot

In [ ]:
def read_dataset(path):
    #read image
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    #normalization
    image = tf.cast(image,tf.float32)/255.
    
    #read label
    class_name = get_class_name(path)
    label = onehot_encoding(class_name)
    
    return image, label


In [ ]:
#이미지를 변환시키는 image preprocess

#image, label = read_dataset(path)
#image.shape, label.shape

In [ ]:
def image_preprocess(image, label):
    #tf.image. 뒤에 제공되는 함수는 매우 많다.

    #이 중 random_flip_left_right 을 한 번 해보면, 
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

In [ ]:
#위의 함수 실행 한 것 시각화 해서 잘 만 들어 졌는지 확인
transformed_image, label = image_preprocess(image, label)
transformed_image.shape

In [ ]:
plt.subplot(1, 2, 1)
plt.title('Original image')
plt.imshow(image)

plt.subplot(1, 2, 2)
plt.title('Transformed image')
plt.imshow(transformed_image)

plt.show() #좌우 반전이 잘 된 것을 알 수 있다. 

In [ ]:
#train_dataset 처리

train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(read_dataset) #image 와 label이 함께 담김

#이때 data generator를 사용하기 위해선 이미지에 변화를 주고 해야 하니transform image가 
train_dataset = train_dataset.map(image_preprocess)

#batch
train_dataset = train_dataset.batch(batch_size)

#shuffle
train_dataset = train_dataset.shuffle(buffer_size = len(train_paths))

#repeat : 계속 돌아가게 하기 위해서
train_dataset = train_dataset.repeat()

In [ ]:
#test_dataset 처리

test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(read_dataset)
test_dataset = test_dataset.map(image_preprocess)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.shuffle(buffer_size = len(test_paths))
test_dataset = test_dataset.repeat()

### 4. Training
앞에서 이미지 preprocessing 다 했으니 이제 본격적인 Training

In [ ]:
model.fit_generator(
    train_dataset,
    steps_per_epoch = len(train_paths)//batch_size, #len(train_dataset)하면 에러남
    validation_data = test_dataset,
    validation_steps = len(test_paths)//batch_size,
    epochs = num_epochs 
)